### Configs

In [1]:
## Importando Pacotes
import time
tic = time.time()

import pyarrow.parquet as pq
import pandas as pd
import numpy as np

## Configs
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 50)

seed = 123

In [2]:
## Importando Dataset Geral
file_path1 = 'dados/df_cashout_limpo.parquet'
df1 = pd.read_parquet(f'{file_path1}')

file_path2 = 'dados/df_cashout_limpo2.parquet'
df2 = pd.read_parquet(f'{file_path2}')

columns_not_in_df1 = df2.columns.difference(df1.columns)

df2['last_status'] = np.where(df2['last_treatment_status'].isin(['CONFIRMED_FRAUD']), 'FRAUD',
                            np.where(df2['last_treatment_status'].isin(['SUSPECTED_FRAUD']), 'SUSPECTED_FRAUD',
                            df2['last_status']))

df2.drop(columns_not_in_df1, axis='columns', inplace=True)
list(df1.columns) == list(df2.columns)

df = pd.concat([df1, df2], ignore_index=True)
df['last_status'].value_counts(dropna=False)

df.sort_values('last_status_changed_at', ascending=False, inplace=True)
df.drop_duplicates(subset='transaction_id', keep='first', inplace=True)
df.duplicated('transaction_id').sum()

0

In [3]:
df['transaction_datetime_request'].describe()

count                         10110814
mean     2024-08-25 09:58:48.006620672
min                2024-01-01 03:00:59
25%      2024-07-19 22:22:02.249999872
50%         2024-08-26 19:02:50.500000
75%      2024-10-02 03:07:51.880999936
max         2024-11-06 22:53:21.075000
Name: transaction_datetime_request, dtype: object

In [4]:
file_path_fraud = 'dados/cashout_fraudes_id.csv'

datatypes = {'CNPJ da conta': 'object',
        'Dados do recebedor - Documento': 'object',
        }

df_fraud = pd.read_csv(file_path_fraud, dtype = datatypes)
df_fraud['fraud'] = 1
df_fraud['Transação finalizada em'] = pd.to_datetime(df_fraud['Transação finalizada em'])

def limpa_campo_moeda(value):
    value = value.replace("R$", "").strip()
    value = value.replace(".", "")
    value = value.replace(",", ".")
    return float(value)

df_fraud['Valor da transação'] = df_fraud['Valor da transação'].apply(limpa_campo_moeda)

df_fraud['Transação finalizada em'] = pd.to_datetime(df_fraud['Transação finalizada em'])
df_fraud[df_fraud['Transação finalizada em'].dt.year == 2024].shape

(111, 18)

In [5]:
df_fraud[df_fraud['Transação finalizada em'] >= '2024-05-01'][['Valor da transação']].sum()

Valor da transação   251,148.07
dtype: float64

In [6]:
df_fraud[['Valor da transação']].sum()

Valor da transação   1,509,329.00
dtype: float64

In [7]:
df_fraud[df_fraud['Transação finalizada em'] >= '2024-05-01'][['CNPJ da conta']].nunique()

CNPJ da conta    9
dtype: int64

In [8]:
df_fraud[['CNPJ da conta']].nunique()

CNPJ da conta    22
dtype: int64

In [9]:
df_fraud['Dados do recebedor - Documento'].unique()

array(['03310768457', '00776574000903', '19037276000172',
       '11037227000128', '17283532000186', '21385258802', '43423241802',
       '52616440000171', '09785780406', '11700719459', '06571971909',
       '58038946804', '58913037874', '55503083000123', '58708580898',
       '55053937800', '52849878000108', '06204581783', '81375808591',
       '06088441798', '08609065740', '42059494000120', '97649520897',
       '49210426000169', '49271808835', '14674796750', '17644085785',
       '15017561765', '12903498784', '24800661000160', '96845694168',
       '03822236322', '23059220867', '28799936000143', '52582356000184',
       '70260107271', '48815566813', '01602265208', '51687384860',
       '31290307865'], dtype=object)

### Analises

In [10]:
for i in [2,3,4,5,6]:
    print(f'Mês: {i} - Casos: {df[df['transaction_datetime_request'] >= f'2024-0{i}-01'].shape[0]}')

Mês: 2 - Casos: 10110809
Mês: 3 - Casos: 10110806
Mês: 4 - Casos: 10110803
Mês: 5 - Casos: 10110799
Mês: 6 - Casos: 10105835


In [11]:
df['transaction_datetime_request'].describe()

count                         10110814
mean     2024-08-25 09:58:48.006620672
min                2024-01-01 03:00:59
25%      2024-07-19 22:22:02.249999872
50%         2024-08-26 19:02:50.500000
75%      2024-10-02 03:07:51.880999936
max         2024-11-06 22:53:21.075000
Name: transaction_datetime_request, dtype: object

In [12]:
## End2end da transação 
## payouts - End2end da transação → payout_id

df_deltatime = pd.concat([df[['transaction_id', 'payer_cnpj', 'transaction_datetime_request', 'transaction_amount_total', 'last_status']].sort_values('transaction_amount_total', ascending=False).reset_index(drop=True), 
            df_fraud[['payouts - End2end da transação → payout_id', 'CNPJ da conta', 'Transação finalizada em', 'Valor da transação']].sort_values('Valor da transação', ascending=False).reset_index(drop=True)], axis='columns').sort_values(['last_status', 'transaction_amount_total'], ascending=False)

df_deltatime['Transação finalizada em'] = pd.to_datetime(df_deltatime['Transação finalizada em'])

df_deltatime['time_delta'] =  df_deltatime['transaction_datetime_request'] - df_deltatime['Transação finalizada em']
df_deltatime

,transaction_id,payer_cnpj,transaction_datetime_request,transaction_amount_total,last_status,payouts - End2end da transação → payout_id,CNPJ da conta,Transação finalizada em,Valor da transação,time_delta
70,1ef8465a-cfc3-64a6-aab8-393909983ac1,31703183000135,2024-10-07 02:13:24.786,"1,300,000.00",SUSPECTED_FRAUD,1ef42b12-233f-6272-bd84-12bba74d69be,46250852000138,2024-07-15 10:50:00,"6,285.31",83 days 15:23:24.786000
92,1ef82626-4860-67bf-849a-53fca02ec892,46919983000165,2024-10-04 15:06:45.946,"1,000,000.00",SUSPECTED_FRAUD,NaN,35304995600,2023-12-04 14:50:00,"1,900.00",305 days 00:16:45.946000
195,1ef82626-48d4-6d06-ac1e-6299a92a39a4,46919983000165,2024-10-04 15:07:12.000,"639,320.00",SUSPECTED_FRAUD,NaN,NaN,NaT,NaN,NaT
3781,1ef825d2-085f-6e2b-ac1e-b39fce7ac55a,44097674000121,2024-10-04 14:27:38.000,"125,057.50",SUSPECTED_FRAUD,NaN,NaN,NaT,NaN,NaT
10100,1ef82611-9a7a-6aae-8df5-91c00eb7f52c,40050970000116,2024-10-04 14:58:17.400,"69,336.00",SUSPECTED_FRAUD,NaN,NaN,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
10110809,E27084098202408282339YyGfwIY6rjJ,14380200000121,2024-08-28 23:39:58.119,0.01,APPROVED,NaN,NaN,NaT,NaN,NaT
10110810,E270840982024082823398CMc3oWEXxH,14380200000121,2024-08-28 23:39:58.224,0.01,APPROVED,NaN,NaN,NaT,NaN,NaT
10110811,E27084098202408282339i4WJSjvMhSN,35410271000108,2024-08-28 23:39:44.935,0.01,APPROVED,NaN,NaN,NaT,NaN,NaT
10110812,E27084098202408282339uk7qEDAFMTo,35410271000108,2024-08-28 23:39:31.964,0.01,APPROVED,NaN,NaN,NaT,NaN,NaT


In [13]:
df_fraud[(df_fraud['Transação finalizada em'] <= '2024-07-24') & (df_fraud['Transação finalizada em'].dt.year == 2024)].head()

,Tipo de MED,Nome da conta,CNPJ da conta,Pedido de infração criado em,End2end da transação,Valor da transação,Transação finalizada em,Dados do recebedor - Nome,Dados do recebedor - Documento,Dados do recebedor - Tipo de conta,Dados do recebedor - Conta,Dados do recebedor - Agência,Dados do recebedor - Código do banco,Dados do recebedor - ISPB do banco,Dados do recebedor - Nome do banco,payouts - End2end da transação → payout_id,payouts - End2end da transação → pix_payout_id,fraud
16,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",E27084098202407151509nZl7NlzPQq7,"3,329.43",2024-07-15 12:09:00,MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bc2-ae31-60b3-b6e7-2d32c36b72cb,1ef42bc2-c182-6d48-b4d7-8509827f7e15,1
17,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",E27084098202407151508c1EuLCg6ojc,326.01,2024-07-15 12:08:00,MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bc0-d264-6e13-b6e7-2e6a24f89cac,1ef42bc0-d960-6417-b4d7-126bc3a47d7c,1
18,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",E27084098202407151507Hr7ST5qJloy,326.04,2024-07-15 12:07:00,MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bbe-972a-695d-b6e7-540f942a4571,1ef42bbe-9e6f-680f-8b45-b30b84025a88,1
19,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",E270840982024071515071I3LHmNhMBT,112.26,2024-07-15 12:07:00,MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bbe-35d3-6a45-bd84-601d726dc913,1ef42bbe-3bd5-68a7-b4d7-7e1ab85e203f,1
20,solicitado,TECNOLOGIA E PAGAMENTOS KING LTDA,46250852000138,"15 Jul, 2024, 14:49",E270840982024071515061H8nM8VqQe3,292.91,2024-07-15 12:06:00,MATHEUS SOUZA DOS SANTOS,11700719459,CONTA_CORRENTE,352449674,352,70.00,208,Banco de Brasília - BRB,1ef42bbd-da7e-6ec9-b6e7-5416ee782d19,1ef42bbd-e093-625e-b4d7-2b22acb492a7,1


In [14]:
df_fraud[(df_fraud['Transação finalizada em'] <= '2024-07-24') & (df_fraud['Transação finalizada em'].dt.year == 2024)]['payouts - End2end da transação → pix_payout_id'].isna().sum()

61

In [15]:
df_fraud_wo_id = df_fraud[df_fraud['payouts - End2end da transação → payout_id'].isnull()]

df_fraud_wo_id['Transação finalizada em'].max()

Timestamp('2024-04-05 21:48:00')

Antes do dia 16 de julho enviamos o payout_id  
Do dia 16 de julho até o dia 24 de julho enviamos pix_payout_id  
Depois do dia 24 somente end2end_id 

In [16]:
## Craindo a coluna de ID consolidade
df['transaction_id'] = df['transaction_id'].apply(
    lambda x: ''.join(str(x).lower().replace('-', '')) if pd.notnull(x) else x
)

df_fraud['payouts - End2end da transação → payout_id'] = df_fraud['payouts - End2end da transação → payout_id'].apply(
    lambda x: ''.join(str(x).lower().replace('-', '')) if pd.notnull(x) else x
)

df_fraud['payouts - End2end da transação → pix_payout_id'] = df_fraud['payouts - End2end da transação → pix_payout_id'].apply(
    lambda x: ''.join(str(x).lower().replace('-', '')) if pd.notnull(x) else x
)

df_fraud['End2end da transação'] = df_fraud['End2end da transação'].apply(
    lambda x: ''.join(str(x).lower().replace('-', '')) if pd.notnull(x) else x
)

def definir_coluna(row):
    if row['Transação finalizada em'] < pd.Timestamp('2024-07-16'):
        return row['payouts - End2end da transação → payout_id']
    elif pd.Timestamp('2024-07-16') <= row['Transação finalizada em'] <= pd.Timestamp('2024-07-24'):
        return row['payouts - End2end da transação → pix_payout_id']
    elif row['Transação finalizada em'] > pd.Timestamp('2024-07-24'):
        return row['End2end da transação']

df_fraud['transaction_id'] = df_fraud.apply(definir_coluna, axis=1)
df_fraud.duplicated('transaction_id').sum()

df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 19 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Tipo de MED                                     133 non-null    object        
 1   Nome da conta                                   133 non-null    object        
 2   CNPJ da conta                                   133 non-null    object        
 3   Pedido de infração criado em                    133 non-null    object        
 4   End2end da transação                            133 non-null    object        
 5   Valor da transação                              133 non-null    float64       
 6   Transação finalizada em                         133 non-null    datetime64[ns]
 7   Dados do recebedor - Nome                       133 non-null    object        
 8   Dados do recebedor - Documento                  13

In [17]:
df_fraud['transaction_id'].isna().sum()

83

In [18]:
# df_fraud[df_fraud['transaction_id'].isna()]

df_fraud['transaction_id'] = np.where(df_fraud['transaction_id'].isna(), df_fraud['End2end da transação'], df_fraud['transaction_id'])
df_fraud['transaction_id'].isna().sum()

0

In [19]:
df_fraud[['Transação finalizada em', 'End2end da transação', 'payouts - End2end da transação → pix_payout_id', 'payouts - End2end da transação → payout_id', 'transaction_id']].sort_values('Transação finalizada em')

,Transação finalizada em,End2end da transação,payouts - End2end da transação → pix_payout_id,payouts - End2end da transação → payout_id,transaction_id
132,2023-11-10 18:02:00,e27084098202311102102zbzot66iz0u,NaN,NaN,e27084098202311102102zbzot66iz0u
131,2023-11-10 18:02:00,e27084098202311102102mkzi1suuqfa,NaN,NaN,e27084098202311102102mkzi1suuqfa
130,2023-11-10 18:02:00,e27084098202311102102666vr8y9drc,NaN,NaN,e27084098202311102102666vr8y9drc
127,2023-11-10 18:03:00,e270840982023111021030dxqdcz5uha,NaN,NaN,e270840982023111021030dxqdcz5uha
129,2023-11-10 18:03:00,e27084098202311102102dtvlvdcbwmp,NaN,NaN,e27084098202311102102dtvlvdcbwmp
...,...,...,...,...,...
4,2024-10-23 18:00:00,e27084098202410232100lw7eohx6azp,1ef9181c895860b0bf64bbd63c144cfb,1ef9181c84e4612f9efd2bd8bfb28ee6,e27084098202410232100lw7eohx6azp
3,2024-10-23 18:10:00,e27084098202410232109m7chsfbvtdy,1ef9183295fb697fbf644c7d52359afe,1ef9183292ff61de9efdeaa460a33460,e27084098202410232109m7chsfbvtdy
2,2024-10-23 18:16:00,e27084098202410232116wlpszekosgq,1ef91841ad1261b5bd45dee57d367b31,1ef91841aa9f68529c6c4941297a38de,e27084098202410232116wlpszekosgq
1,2024-10-23 18:18:00,e27084098202410232118z7rokbwv6z4,1ef91844d3a46c659173578997c9838e,1ef91844d0e06bdc9efd2f20c625029b,e27084098202410232118z7rokbwv6z4


In [20]:
df_merged = df.merge(df_fraud[['transaction_id', 'fraud']], left_on='transaction_id', right_on='transaction_id', how='left')

f"""
{df.shape}
{df_fraud.shape}
{df_merged.shape}
"""

'\n(10110814, 57)\n(133, 19)\n(10110814, 58)\n'

ONDE ESTÃO ESSES 62 CASOS ????
1. casos que não estão na nossa base (barte)
2. ID errado 
3. 

In [21]:
print(df_merged['fraud'].sum())
pd.crosstab(df_merged['last_status'], df_merged['fraud'], dropna=False) 

49.0


fraud,1.00,NaN
last_status,,
APPROVED,14,10092969
FRAUD,35,0
REJECTED,0,17541
SUSPECTED_FRAUD,0,255


In [22]:
111 - 49

62

In [23]:
not_found = df_fraud[~df_fraud['transaction_id'].isin(df_merged['transaction_id'])]
not_found.shape

(84, 19)

In [24]:
# not_found.to_parquet('dados/fraud_not_found.parquet')

In [25]:
df_fraud_wo_id.sort_values('Valor da transação', ascending=True).head(5)

,Tipo de MED,Nome da conta,CNPJ da conta,Pedido de infração criado em,End2end da transação,Valor da transação,Transação finalizada em,Dados do recebedor - Nome,Dados do recebedor - Documento,Dados do recebedor - Tipo de conta,Dados do recebedor - Conta,Dados do recebedor - Agência,Dados do recebedor - Código do banco,Dados do recebedor - ISPB do banco,Dados do recebedor - Nome do banco,payouts - End2end da transação → payout_id,payouts - End2end da transação → pix_payout_id,fraud
105,solicitado,BARTE BRASIL LTDA.,44522660000108,"28 Feb, 2024, 16:52",E27084098202402051753u6HYyBc0Icq,47.81,2024-02-05 14:53:00,JS NEGOCIOS IRACEMA,24800661000160,CONTA_PAGAMENTO,1809725,1,403.00,37880206,Cora SCD S.A.,NaN,NaN,1
117,solicitado,TNL INTERNET LTDA,27692571000190,"24 Nov, 2023, 09:59",E27084098202311221252kAFbKRIcvUP,85.40,2023-11-22 09:52:00,DEUSIMAR VIEIRA DE SOUSA,28799936000143,CONTA_CORRENTE,8822813098,1,323.00,10573521,Mercado Pago,NaN,NaN,1
54,solicitado,Aiqfome LTDA,09186786000158,"26 Mar, 2024, 16:17",E27084098202403260318txa55zdjNbg,150.00,2024-03-26 00:18:00,Vera Lucia Rodrigues Zonzini,97649520897,CONTA_PAGAMENTO,18310862,1,301.00,13370835,BPP Instituição de Pagamento S.A.,NaN,NaN,1
112,solicitado,PAULO ROBERTO DE OLIVEIRA SCHMIDT,35304995600,"19 Jan, 2024, 19:18",E27084098202312041957DKokhYOdqp3,170.00,2023-12-04 17:04:00,Rodrigo Catulino dos Santos,23059220867,CONTA_POUPANCA,49644367,1,260.00,18236120,Nu Pagamentos S.A.,NaN,NaN,1
124,solicitado,BRUNO S. IKEJIRI DIFERENTE EVENTOS,17718506000133,"16 Nov, 2023, 14:41",E27084098202311142131LsNNPzbX0Tj,220.00,2023-11-14 18:31:00,JHONATAN PEREIRA BARCELOS,70260107271,CONTA_PAGAMENTO,17339086,1,290.00,8561701,Pagseguro Internet S.A,NaN,NaN,1


In [26]:
not_found.sort_values('Valor da transação', ascending=True).tail(5)

,Tipo de MED,Nome da conta,CNPJ da conta,Pedido de infração criado em,End2end da transação,Valor da transação,Transação finalizada em,Dados do recebedor - Nome,Dados do recebedor - Documento,Dados do recebedor - Tipo de conta,Dados do recebedor - Conta,Dados do recebedor - Agência,Dados do recebedor - Código do banco,Dados do recebedor - ISPB do banco,Dados do recebedor - Nome do banco,payouts - End2end da transação → payout_id,payouts - End2end da transação → pix_payout_id,fraud,transaction_id
87,solicitado,"Perfect Pay Tecnologia, Servicos e Intermediac...",28059320000136,"19 Apr, 2024, 14:45",e27084098202402161312gknawkfdadz,"37,351.87",2024-02-16 10:12:00,49.210.426 EDVALSON RODRIGUES CHAVES JUNIOR,49210426000169,CONTA_PAGAMENTO,92688372,1,260.00,18236120,Nu Pagamentos S.A.,NaN,NaN,1,e27084098202402161312gknawkfdadz
70,solicitado,"Perfect Pay Tecnologia, Servicos e Intermediac...",28059320000136,"19 Apr, 2024, 14:44",e27084098202403041355syijhbwcnqy,"37,372.85",2024-03-04 10:55:00,RAYRA OLIVEIRA SILVA 02113719630,42059494000120,CONTA_PAGAMENTO,11687989,1,260.00,18236120,Nu Pagamentos S.A.,NaN,NaN,1,e27084098202403041355syijhbwcnqy
79,solicitado,Tradio Bank LTDA,36759990000193,"21 Feb, 2024, 18:12",e27084098202402211616urpmfydjqux,"41,000.00",2024-02-21 13:18:00,Vera Lucia Terra,96845694168,CONTA_PAGAMENTO,1952840207,1,323.00,10573521,Mercado Pago,NaN,NaN,1,e27084098202402211616urpmfydjqux
69,solicitado,"Perfect Pay Tecnologia, Servicos e Intermediac...",28059320000136,"19 Apr, 2024, 14:54",e27084098202403061815tahpqk9p7ru,"41,845.29",2024-03-06 15:15:00,RAYRA OLIVEIRA SILVA 02113719630,42059494000120,CONTA_PAGAMENTO,11687989,1,260.00,18236120,Nu Pagamentos S.A.,NaN,NaN,1,e27084098202403061815tahpqk9p7ru
53,solicitado,"Perfect Pay Tecnologia, Servicos e Intermediac...",28059320000136,"19 Apr, 2024, 14:44",e2708409820240402144859z7o9ifexb,"70,362.60",2024-04-02 11:48:00,RAYRA OLIVEIRA SILVA 02113719630,42059494000120,CONTA_PAGAMENTO,11687989,1,260.00,18236120,Nu Pagamentos S.A.,NaN,NaN,1,e2708409820240402144859z7o9ifexb


In [27]:
# & (df['transaction_amount_total'] == 1004.13)
df[(df['payer_cnpj'] == '44522660000108')].sort_values(['transaction_datetime_request'], ascending=True)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,transaction_receiver_banking_data_bank_code,transaction_receiver_pix_key_type,transaction_receiver_pix_key,approved_at,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl
24004,2024-06-04 13:56:19.334071,e5a97044-b990-5ed8-9254-8098dd42c006,transfeera,1ef227a37c5e6eceb1bdb7869432e5b9,,BRL,"16,335.48",2024-06-04 13:56:18.057,cash-out,pix,e3c0004e-509a-45e2-8752-3185f7412c29,BARTE BRASIL LTDA.,,44522660000108,01310200,São Paulo,AV PAULISTA,1636,SALA 1504,BR,None,None,[],[],None,...,None,[PHONE],[+5511988061019],2024-06-04 13:56:19.334385,NaT,ALGORITHM,NaT,None,APPROVED,2024-06-04 13:56:19.334385,ALGORITHM,[],None,None,LOW,1,cnpj,44522660000108,None,None,None,None,None,None,NaN
24005,2024-06-04 13:56:19.969347,82a3990c-b00e-5595-89a9-da19e663364b,transfeera,1ef227a3835d6305b1bdff9319687e12,,BRL,"10,331.25",2024-06-04 13:56:18.851,cash-out,pix,e3c0004e-509a-45e2-8752-3185f7412c29,BARTE BRASIL LTDA.,,44522660000108,01310200,São Paulo,AV PAULISTA,1636,SALA 1504,BR,None,None,[],[],None,...,None,[CNPJ],[51356237000140],2024-06-04 13:56:19.969747,NaT,ALGORITHM,NaT,None,APPROVED,2024-06-04 13:56:19.969747,ALGORITHM,[],None,None,LOW,1,cnpj,44522660000108,None,None,None,None,None,None,NaN
24008,2024-06-04 13:56:20.932309,7cb1342c-5569-5c6d-8b61-683004e1a454,transfeera,1ef227a3862d6b8db1bdfaca1360d907,,BRL,"23,338.65",2024-06-04 13:56:18.924,cash-out,pix,e3c0004e-509a-45e2-8752-3185f7412c29,BARTE BRASIL LTDA.,,44522660000108,01310200,São Paulo,AV PAULISTA,1636,SALA 1504,BR,None,None,[],[],None,...,None,[CNPJ],[25018028000187],2024-06-04 13:56:20.932611,NaT,ALGORITHM,NaT,None,APPROVED,2024-06-04 13:56:20.932611,ALGORITHM,[],None,None,LOW,1,cnpj,44522660000108,None,None,None,None,None,None,NaN
23966,2024-06-04 13:56:50.127136,d746eaac-d0bc-5e33-af0c-7bfe87021a9b,transfeera,1ef227a4a2ac66a1b1bd6b11cfa7fc68,,BRL,282.86,2024-06-04 13:56:18.932,cash-out,pix,e3c0004e-509a-45e2-8752-3185f7412c29,BARTE BRASIL LTDA.,,44522660000108,01310200,São Paulo,AV PAULISTA,1636,SALA 1504,BR,None,None,[],[],None,...,None,[CNPJ],[73746414000138],2024-06-04 13:56:50.127419,NaT,ALGORITHM,NaT,None,APPROVED,2024-06-04 13:56:50.127419,ALGORITHM,[],None,None,LOW,1,cnpj,44522660000108,None,None,None,None,None,None,NaN
23969,2024-06-04 13:56:50.990070,e4ee5acf-1791-53c9-a8c0-16a5cf51ad3b,transfeera,1ef227a4a3c561b8b1bd03e533dd71e3,,BRL,"56,938.29",2024-06-04 13:56:18.941,cash-out,pix,e3c0004e-509a-45e2-8752-3185f7412c29,BARTE BRASIL LTDA.,,44522660000108,01310200,São Paulo,AV PAULISTA,1636,SALA 1504,BR,None,None,[],[],None,...,None,[CNPJ],[27759269000102],2024-06-04 13:56:50.990384,NaT,ALGORITHM,NaT,None,APPROVED,2024-06-04 13:56:50.990384,ALGORITHM,[],None,None,LOW,1,cnpj,44522660000108,None,None,None,None,None,None,NaN
...,

In [28]:
# & (df['transaction_amount_total'] == 1004.13)
df[(df['payer_cnpj'] == '44522660000108') & (df['transaction_datetime_request'] <= '2024-03-01') & (df['transaction_amount_total'] >= 1001.) & (df['transaction_amount_total'] <= 1010.)].sort_values(['transaction_amount_total'], ascending=True)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,transaction_receiver_banking_data_bank_code,transaction_receiver_pix_key_type,transaction_receiver_pix_key,approved_at,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl


In [29]:
df[(df['payer_cnpj'] == '27692571000190') & (df['transaction_amount_total'] == 85.40)].sort_values('transaction_amount_total', ascending=False).head(15)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,transaction_receiver_banking_data_bank_code,transaction_receiver_pix_key_type,transaction_receiver_pix_key,approved_at,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl
2351790,2024-07-17 12:24:40.529385,3e02177d-77d7-58b0-beec-177aa0d212d6,transfeera,1ef443788bfc60aba7547ee00c7b2d06,,BRL,85.40,2024-07-17 12:08:22.688,cash-out,pix,cf933a48-d98f-4e34-94f6-7c42d9300daf,TNL INTERNET LTDA,44997470240,27692571000190,77015018,Palmas,Quadra ACSO 1 Rua SO 5,18,Sala 1,BR,CHECKING_ACCOUNT,593,[],[],46413537191,...,104,[],[],2024-07-17 12:24:40.529536,NaT,ALGORITHM,NaT,None,APPROVED,2024-07-17 12:24:40.529536,ALGORITHM,[],None,None,None,1,cnpj,27692571000190,cpf,46413537191,None,None,None,None,NaN


In [30]:
df[(df['payer_cnpj'] == '36759990000193') & (df['transaction_amount_total'] == 41000)].sort_values('transaction_amount_total', ascending=False).head(15)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,transaction_receiver_banking_data_bank_code,transaction_receiver_pix_key_type,transaction_receiver_pix_key,approved_at,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl
1028655,2024-06-24 20:02:59.795260,4dcb6c83-59ec-5f26-9553-d32b314ffd8b,transfeera,1ef3264c17126fef9a2cc96feacbbca2,Enviado por Tradio Bank,BRL,"41,000.00",2024-06-24 20:02:58.565,cash-out,pix,ac60d9e6-8ab9-4ccd-8cad-631351b35571,Tradio Bank LTDA,62985091933,36759990000193,74280010,Goiania,Rua C-255,400,"Sala 503, Eldorado Business Tower",BR,None,593,[],[],None,...,None,[CNPJ],[54725763000129],2024-06-24 20:02:59.795402,NaT,ALGORITHM,NaT,None,APPROVED,2024-06-24 20:02:59.795402,ALGORITHM,[],None,None,None,1,cnpj,36759990000193,None,None,None,None,None,None,NaN


In [31]:
file_path = 'dados/df_cashout_limpo.parquet'
df_search = pd.read_parquet(f'{file_path}')

df_search[(df_search['transaction_payer_registration_data_doc'] == '28059320000136') & (df_search['transaction_amount_total'] > 31990)].sort_values('transaction_amount_total', ascending=True).head(2)

,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,transaction_receiver_banking_data_bank_code,transaction_receiver_pix_key_type,transaction_receiver_pix_key,approved_at,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl
7395330,2024-08-21 14:18:31.029528,00fcc19b-76c1-5ff6-9639-a2a4f0320245,transfeera,E27084098202408211418f19MeHsLnZR,Pagamento Perfect Pay,BRL,"31,994.10",2024-08-21 14:18:29,cash-out,pix,7c6ed037-2986-4f8f-90bb-a543f549e647,PERFECT PAY,2421091185,28059320000136,06460040,BARUERI,AV MARCOS PENTEADO DE ULHOA RODRIGUES,939,ANDAR 8 TORRE 1 EDIF JACARANDA,BR,CHECKING_ACCOUNT,593,[],[],28971170000132,...,336,[RANDOM],[2a0097e7-6aeb-43a5-93ac-17cf991dbba9],2024-08-21 14:18:31.029787,NaT,ALGORITHM,NaT,None,APPROVED,2024-08-21 14:18:31.029787,ALGORITHM,[],None,None,None,1,cnpj,28059320000136,cnpj,28971170000132,{},None,None,null,"31,994.10"
1789052,2024-07-08 16:45:18.073110,d7423e2c-f75c-5cab-b835-1ffba085deda,transfeera,1ef3d497-514d-6382-9574-deaa4d804a2b,Pagamento Perfect Pay,BRL,"31,994.10",2024-07-08 16:45:17,cash-out,pix,7c6ed037-2986-4f8f-90bb-a543f549e647,PERFECT PAY,2421091185,28059320000136,06460040,BARUERI,AV MARCOS PENTEADO DE ULHOA RODRIGUES,939,ANDAR 8 TORRE 1 EDIF JACARANDA,BR,CHECKING_ACCOUNT,593,[],[],50889746000176,...,None,[CNPJ],[50889746000176],2024-07-08 16:45:18.073282,NaT,ALGORITHM,NaT,None,APPROVED,2024-07-08 16:45:18.073282,ALGORITHM,[],None,None,None,1,cnpj,28059320000136,cnpj,50889746000176,None,None,None,None,NaN


In [32]:
print(df_merged.shape)
df_merged[df_merged['fraud'] == 1]

(10110814, 58)


,internal_created_at,internal_id,client_name,transaction_id,transaction_description,transaction_currency_code,transaction_amount_total,transaction_datetime_request,transaction_type,transaction_channel,transaction_payer_id,transaction_payer_registration_data_name,transaction_payer_registration_data_phone_number,payer_cnpj,transaction_payer_registration_data_address_street_number,transaction_payer_registration_data_address_city,transaction_payer_registration_data_address_state,transaction_payer_registration_data_address_street_name,transaction_payer_registration_data_address_complement,transaction_payer_registration_data_address_country,transaction_payer_banking_data_account_type,transaction_payer_banking_data_bank_code,transaction_payer_pix_key_type,transaction_payer_pix_key,transaction_receiver_id,...,transaction_receiver_pix_key_type,transaction_receiver_pix_key,approved_at,rejected_at,processed_by,fraud_detected_at,fraud_detected_by,last_status,last_status_changed_at,last_status_changed_by,rules_triggered,reviewer_name,reviewer_email,risk,transaction_instalments,transaction_payer_registration_data_doc_type,transaction_payer_registration_data_doc,transaction_receiver_registration_data_doc_type,transaction_receiver_registration_data_doc,extra_data,transaction_payer_banking_data_account_id,transaction_receiver_banking_data_account_id,order_items,transaction_amount_total_brl,fraud
832184,2024-10-25 12:00:13.741438,11e6db10-ad00-58e5-9a0c-bb03fdfeec7e,transfeera,e27084098202410251200e6ij18sv6nq,,BRL,"3,000.00",2024-10-25 12:00:12.336,cash-out,pix,1ef8fb3e-af59-67e6-bafd-146ade6fa633,VARGAS TECNOLOGIA E VENDAS ONLINE,,04721736000100,85906370,Toledo,AV JOSE JOAO MURARO,1059,,BR,CHECKING_ACCOUNT,593,[],[],03310768457,...,[EMAIL],[maurogomesbarbosa463@gmail.com],2024-10-25 12:00:13.741676,NaT,RULE,NaT,None,APPROVED,2024-10-25 12:00:13.741676,RULE,[956],None,None,LOW,1,cnpj,04721736000100,cpf,03310768457,"{""operation_type"": ""transfer"", ""source"": ""API""}",a36b5265-b263-5f91-a46d-5d679100e6ef,106f2fc8-5033-5717-9e0b-c026f34f4791,null,"3,000.00",1.00
912794,2024-10-23 21:18:09.575057,25818f15-11a0-56f7-a683-6eb6eb193ef2,transfeera,e27084098202410232118z7rokbwv6z4,,BRL,"2,310.00",2024-10-23 21:18:07.881,cash-out,pix,4288df53-6a28-4003-a1d3-6dc00e6ac656,PIX GLOBAL,,45810578000141,05616011,SÃO PAULO,R DOM ARMANDO LOMBARDI,800,CONJ 75S - SALA 8,BR,CHECKING_ACCOUNT,593,[],[],00776574000903,...,[RANDOM],[94dd41f2-c1bc-4ad4-aa2d-5a8b9117fb71],2024-10-23 21:18:09.575294,NaT,RULE,2024-10-24 17:34:15.286814,MANUAL,FRAUD,2024-10-23 21:18:09.575294,RULE,[956],Marcos Machioni,marcos.machioni@transfeera.com,LOW,1,cnpj,45810578000141,cnpj,00776574000903,"{""operation_type"": ""transfer"", ""source"": ""API""}",2f4e866c-cb1c-57de-b0a6-00874bbe09a6,ba7b874c-1a3d-51f5-b2a1-7adce7cd69f9,null,"2,310.00",1.00
912884,2024-10-23 21:16:45.183418,4b40d85f-37fc-53fe-afa8-1d51b453241c,transfeera,e27084098202410232116wlpszekosgq,,BRL,"9,200.00",2024-10-23 21:16:43.242,cash-out,pix,4288df53-6a28-4003-a1d3-6dc00e6ac656,PIX GLOBAL,,45810578000141,05616011,SÃO PAULO,R DOM ARMANDO LOMBARDI,800,CONJ 75S - SALA 8,BR,CHECKING_ACCOUNT,593,[],[],00776574000903,...,[RANDOM],[94dd41f2-c1bc-4ad4-aa2d-5a8b9117fb71],2024-10-23 21:16:45.183710,NaT,RULE,2024-10-24 17:39:13.178917,MANUAL,FRAUD,2024-10-23 21:16:45.183710,RULE,[956],Marcos Machioni,marcos.machioni@transfeera.com,LOW,1,cnpj,45810578000141,cnpj,00776574000903,"{""operation_type"": ""transfer"", ""source"": ""API""}",2f4e866c-cb1c-57de-b0a6-00874bbe09a6,ba7b874c-1a3d-51f5-b2a1-7adce7cd69f9,null,"9,200.00",1.00
913776,2024-10-23 21:09:59.781000,f06fd45b-04ad-5967-ba3c-1b2b71782dc2,transfeera,e27084098202410232109m7chsfbvtdy,,BRL,"6,000.00",2024-10-23 21:09:57.997,cash-out,pix,4288df53-6a28-4003-a1d3-6dc00e6ac656,PIX GLOBAL,,45810578000141,05616011,SÃO PAULO,R DOM ARMANDO LOMBARDI,800,CONJ 75S - SALA 8,BR,CHECKING_ACCOUNT,593,[],[],00776574000903,...,[RANDOM],[94dd41f2-c1bc-4ad4-aa2d-5a8b9117fb71],2024-10-23 21:09:5

### Salvando o DF

In [33]:
df_merged.to_parquet('dados/df_merged.parquet')

In [34]:
from time import *

toc = time()

def convert_seconds(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

time_format = convert_seconds(toc-tic)

print(f'Tempo de Processamento: {time_format}')

Tempo de Processamento: 00:03:08
